In [ ]:
# Define other parameters
dt = 600.0
timesteps = 10000
num_test_particles = 10000
saved_points_modularity = 100
skipped_timesteps = 0
inner_radius = 70000 * 1000.0 # Inner boundary of the ring (meters)
outer_radius = 140000 * 1000.0 # Outer boundary of the ring (meters)
output_filename = ".\simulation_test10.bin"
include_particle_moon_collisions = True

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as anim
import numpy as np
import SaturnMoonLibrary as sml
import SaturnMoonLibrary.SaturnMoonSimulation as sms
%matplotlib qt


##Initial Values
num_bodies = 10020   # Number of celestial bodies (adjust based on your setup)
n = 10 # Number of farthest moons to filter out
coords =  [0,1]
moon_count = 20
Filename = "simulation 2025-01-09_16-17-43"


##The Rest
class CelestialBody:
    def __init__(self, name, color, pos,trail=False):
        self.name = name
        self.color = color
        self.pos = pos
        self.trail= trail

def read_binary_file(filename, num_bodies):
    data = np.fromfile(filename, dtype=np.float64)
    positions = data.reshape(-1, num_bodies, 6)
    return positions

# Load the positions array as shown previously
positions = sms.read_binary_file('SaturnModelDatabase/simulation_data/'+Filename+'.bin')
num_bodies=len(positions[0,:,0])
# Debugging: Print the shape of the positions array
print("Loaded positions shape:", positions.shape)

bodies_names = ["Saturn", "Mimas", "Enceladus", "Tethys", "Dione", "Rhea", "Titan", "Hyperion", "Iapetus", "Phoebe", "Janus", "Epimetheus", "Helene", "Telesto", "Calypso", "Atlas", "Prometheus", "Pandora", "Pan","Daphnis"]+list(str(i) for i in range(num_bodies))
colors = ['yellow', 'red', 'chartreuse', 'lightblue', 'orange', 'brown', 'blue', 'pink', 'red', 'black', 'green', 'purple', 'cyan', 'magenta', 'gold', 'silver', 'lime', 'navy', 'maroon','crimson']+num_bodies*["navy"]
tails= [True]*moon_count+[False]*num_bodies
rocks = [CelestialBody(bodies_names[i], colors[i], positions[:, i,:3],tails[i]) for i in range(num_bodies)]

rocks1 = rocks[:moon_count]
nth_largest_indices = np.argsort([np.min(body.pos[:, 0]**2 + body.pos[:, 1]**2) for body in rocks1])[:-n]
rocks2 = [body for i, body in enumerate(rocks1) if i in nth_largest_indices]
Stones = rocks[moon_count:]



# Set up the plots
figure, (ax, ax2) = plt.subplots(1, 2, figsize=(12, 6))

xmax, xmin = np.max([np.max(body.pos[:, coords[0]]) for body in rocks1]), np.min([np.min(body.pos[:, coords[0]]) for body in rocks1])
ymax, ymin = np.max([np.max(body.pos[:, coords[1]]) for body in rocks1]), np.min([np.min(body.pos[:, coords[1]]) for body in rocks1])

# Debugging: Print the min and max values
print(f"xmin: {xmin}, xmax: {xmax}, ymin: {ymin}, ymax: {ymax}")

# Set limits for the first subplot (global view)
ax.set_xlim(1 * xmin - .1 * abs(xmin), xmax + .1 * abs(xmax))
ax.set_ylim(1 * ymin - .1 * abs(ymin), ymax + .1 * abs(ymax))
ax.set_aspect('equal', adjustable='box')

# Filter out n farthest moons for zoomed-in view
nth_largest_indices = np.argsort([np.min(body.pos[:, 0]**2 + body.pos[:, 1]**2) for body in rocks1])[:-n]
rocks2 = [body for i, body in enumerate(rocks1) if i in nth_largest_indices]

xmax2, xmin2 = np.max([np.max(body.pos[0,coords[0]]) for body in rocks2]), np.min([np.min(body.pos[0,coords[0]]) for body in rocks2])
ymax2, ymin2 = np.max([np.max(body.pos[0, coords[1]]) for body in rocks2]), np.min([np.min(body.pos[0, coords[1]]) for body in rocks2])
xmax2, xmin2 = -3e8,3e8
ymax2, ymin2 = -3e8,3e8

ax2.set_xlim(xmin2, xmax2)
ax2.set_ylim(ymin2, ymax2)
ax2.set_aspect('equal', adjustable='box')
# Plotting lines for both subplots
lines = {body: ax.plot(body.pos[0, coords[0]], body.pos[0, coords[1]], marker="o", markevery=[-1], label=f"{body.name}", color=body.color)[0] for body in rocks1}
zoom_lines = {body: ax2.plot(body.pos[0, coords[0]], body.pos[0, coords[1]], marker="o", markevery=[-1], label=f"{body.name}", color=body.color,markersize=5+10*(body.name=='Saturn'))[0] for body in rocks2}
zoom_lines['Ring']= ax2.plot(positions[0,moon_count:,coords[0]], positions[0,moon_count:,coords[1]], ".", label=f"", color="navy",markersize=1)[0]
# ax.legend()
# ax2.legend()
big_traillength=100 #how far ago into the past trail is plotted for big picture
small_traillength=3 #same but for small picture
small_skips=0
big_skips=0

def update(i):
    for body in rocks1:
        #Update the full trajectory plot
       lines[body].set_data(body.pos[max(i-big_traillength*body.trail, 0):i:(big_skips+1), coords[0]], body.pos[max(i-big_traillength*body.trail, 0):i:(big_skips+1), coords[1]])

    for body in rocks2:
        # Update the zoomed-in plot, centered on Saturn
        zoom_lines[body].set_data(
            body.pos[max(i-small_traillength-1, 0):i:(small_skips+1), coords[0]],
            body.pos[max(i-small_traillength-1, 0):i:(small_skips+1), coords[1]]
        )
    zoom_lines['Ring'].set_data(positions[i,moon_count:,coords[0]], positions[i,moon_count:,coords[1]])
    return lines.values(), zoom_lines.values()
    #zoom_lines = dic[i]

#Animation setup
animation = anim.FuncAnimation(
    figure,
    func=update,
    frames=np.arange(0, positions.shape[0], 1),  # Reduce frame step for smoother animation
    interval=1,  # Increase interval time to reduce flashing
    # blit=True
)

labels = {0:'X (m)',1:'Y (m)',2:'Z (m)'}
# Show the animation
plt.legend(loc="upper right")
plt.xlabel(labels[coords[0]])
plt.ylabel(labels[coords[1]])
plt.show()
print(np.shape(positions))

Loaded positions shape: (21, 10020, 6)
xmin: -9792599639.408022, xmax: 3446821673.116924, ymin: -3428592723.663769, ymax: 5671838045.832366
(21, 10020, 6)


In [5]:
#[i[0:3] for i in positions[0]]
positions

array([[[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00,
          1.00000000e-01,  1.00000000e-01,  1.00000000e-01],
        [-2.89185871e+06, -1.35540155e+08,  0.00000000e+00,
          7.01259964e-10, -1.51954881e-11,  0.00000000e+00],
        ...,
        [ 1.35769673e+08,  1.25581307e+07,  0.00000000e+00,
         -6.47003485e-11,  6.96407893e-10,  0.00000000e+00],
        [-9.83461240e+07, -8.90882435e+07,  0.00000000e+00,
          4.75932656e-10, -5.25812092e-10,  0.00000000e+00],
        [ 5.68883519e+07,  1.01836384e+08,  0.00000000e+00,
         -6.60273533e-10,  3.68991628e-10,  0.00000000e+00]],

       [[-5.21772073e+07,  1.11963662e+08,  0.00000000e+00,
         -6.66366741e-10, -3.10591625e-10,  0.00000000e+00],
        [ 4.25193022e+07, -1.24462593e+08,  0.00000000e+00,
          6.74326161e-10,  2.30509348e-10,  0.00000000e+00],
        [-9.81832

In [30]:
plt.plot(positions[800,19:,0],positions[800,19:,1],'.')
plt.xlim(1e12-100,1e12+100)
plt.ylim(1e12-100,1e12+100)
plt.show()
print(min(positions[:,19:,0]**2+positions[:,19:,1]**2+positions[:,19:,2]**2))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [99]:
mass = [5.6834e+26,3.7493e+19,1.08022e+20,6.17449e+20,1.095452e+21,2.306518e+21,1.3452e+23,5.62e+18,1.805635e+21,8.292e+18,1.898e+18,5.264e+17,1.2e+17,4.1e+16,4e+16,7e+16,1.6e+17,1.4e+17,4950000000000000.0]+[1000]*(num_bodies-19)
sample = np.arange(0,1,1)*100
for i,j in enumerate(rocks):
    j.mass=mass[i]
for i,j in enumerate(rocks):
    j.vs = positions[sample,i,3:]
#checking energy conservation
potential=np.zeros(len(rocks[0].pos[sample]))
for p1 in rocks:
    for p2 in rocks:
        if p1==p2: continue
        potential+=-G*p1.mass*p2.mass/np.sqrt(np.sum((p1.pos[sample]-p2.pos[sample])**2,axis=1))
print(potential)
total=(np.sum([it.mass*np.sum(it.vs[0]**2) for it in rocks]))+potential[1]
plt.plot(((np.sum([it.mass*np.sum(it.vs**2,axis=1) for it in rocks],axis=0)+potential)/total-1))
np.average((np.sum([it.mass*np.sum(it.vs**2,axis=1) for it in rocks],axis=0)+potential)/total-1)


KeyboardInterrupt: 

In [67]:
mass = [5.6834e+26,3.7493e+19,1.08022e+20,6.17449e+20,1.095452e+21,2.306518e+21,1.3452e+23,5.62e+18,1.805635e+21,8.292e+18,1.898e+18,5.264e+17,1.2e+17,4.1e+16,4e+16,7e+16,1.6e+17,1.4e+17,4950000000000000.0]
print(len(mass))

{ 
        {"Saturn", 5.6834e+26, {-269883.9483281949, -135765.3275027061, -3951.650821157909}, {0.5711871802735469, -1.2013243498509103, -0.002649024598310003}},
        {"Mimas", 3.7493e+19, {81035480.29437092, 169312183.95669305, -5008234.138896362}, {-12644.305263083728, 6297.454455130696, 88.94865309823108}},
        {"Enceladus", 1.08022e+20, {74430757.24220988, -226809447.4342958, 21407.96787442675}, {11952.422868086827, 3987.846912293124, 1.1428278693433298}},
        {"Tethys", 6.17449e+20, {-288242779.5160179, 62383072.15644543, -3294095.0405105953}, {-2404.960997802739, -11092.403702681071, -175.5975805289068}},
        {"Dione", 1.095452e+21, {371641004.9716364, -60467557.37795043, 151478.8530827912}, {1623.8858533003558, 9912.09784340947, 2.33953748248941}},
        {"Rhea", 2.306518e+21, {434560268.13074356, -297727746.0997345, 405640.78039853484}, {4799.851687334527, 6998.352693285457, -45.38294171281268}},
        {"Titan", 1.3452e+23, {1086893364.6233644, 595435775.6897647, 8662804.048128707}, {-2517.784610306088, 4879.454019893294, 2.9731054586928454}},
        {"Hyperion", 5.62e+18, {1551547985.2923973, 551068256.5099872, 27873784.42066735}, {-1588.2826991700515, 4241.085679421853, -23.838945983885132}},
        {"Iapetus", 1.805635e+21, {3233616105.112864, -1180265742.6553144, 580586590.5193009}, {1064.9562402447725, 3068.929312889123, 727.1400292602492}},
        {"Phoebe", 8.292e+18, {10493204641.068493, -9269949445.322527, 4003120271.5622325}, {-9.35304369489992, -10.13560250729897, -6.107778419356851}},
        {"Janus", 1.898e+18, {5455192.547329088, 150891766.0493537, 401842.6809074324}, {-15880.695030661967, 497.75222521384626, -16.89265298418603}},
        {"Epimetheus", 5.264e+17, {-146788516.48500454, -37298938.402314104, -3226.42274479062}, {4049.5820799759326, -15365.690628725504, -98.31555878873203}},
        {"Helene", 1.2e+17, {196940796.01093596, 322962498.49218494, -1347102.325590057}, {-8498.143863781717, 5271.275272149338, -12.908892995047088}},
        {"Telesto", 4.1e+16, {-202803827.73178875, -214223323.44167382, -4969613.468651005}, {8242.722083970215, -7801.072299848389, 134.90640870442212}},
        {"Calypso", 4e+16, {-100508076.70166014, 276851769.3733176, 7503530.023907208}, {-10675.468332443816, -3857.136633339224, -68.03409271147932}},
        {"Atlas", 7e+16, {-128673364.79805952, 49793594.863064244, -1925.4987124569502}, {-6040.649545599975, -15490.829128297695, 0.7571902948546616}},
        {"Prometheus", 1.6e+17, {-35648607.88915291, 134986140.2751371, -11571.395730172431}, {-15958.481315616384, -4190.175771237246, -2.160717511145594}},
        {"Pandora", 1.4e+17, {141695221.40422508, -6526549.912053117, -125802.30356671562}, {684.4366521167876, 16335.774578561928, 2.4595476948930393}},
        {"Pan", 4950000000000000.0, {-133854142.19770378, -174124.37104152393, -3951.6508211591336}, {5.520894900147233, -16894.5894554284, -0.002649024598247031}},
    }

19


TypeError: unhashable type: 'set'